In [1]:
import matplotlib.pyplot as plt
import numpy as np

0.4066(98)

In [11]:
import re  
import pprint

def parse_latex_table(file_name):
    with open(file_name, "r") as f:
        content = f.read()

    # tables = content.split("\n\n")
    tables = re.split(r'\\end{tabular}', content)[:-1]

    data = {}
    for table in tables:
        lines = table.strip().split("\n")
        if len(lines) < 4:  # If there are not enough lines, skip this table
            continue
        ensemble = lines[2].split("&")[0].strip().replace('$_','_')
        data[ensemble] = {}

        for line in lines[3:-1]:  # Skip the first two lines (headers) and the last line (end of the table)
            cols = line.split("&")

            person = cols[0].strip()
            results = [tuple(map(float, re.findall(r"([\d.]+)", item))) for item in cols[1:]]
            data[ensemble][person] = dict(zip(["N_E0", "Xi_EO", "Sigma_E0", "Lambda_E0", "Xi*_E0", "Delta_E0", "Sigma*_E0"], results))

    return data

file_name = "tables.txt"
data = parse_latex_table(file_name)
pprint.pprint(data)



['\\begin{tabular}{|c|c|c|c|c|c|c|c|c|}', '    \\hline', '    a06m220L & $N_{E0}$ & $\\Xi_{EO}$ & $\\Sigma_{E0}$ & $\\Lambda_{E0}$ & $\\Xi^*_{E0}$ & $\\Delta_{E0}$ & $\\Sigma^*_{E0}$ \\\\', '    \\hline', '    Amy & {0.2925 (41)}   &   { 0.38361 (98)} & {0.3538 (22)}   & { 0.3335 (19)}   & {0.4541 (30)}  & { 0.3943 (90)} & {0.4239 (54)} \\\\', '    \\hline', '    Grant & 0.2976(21) & 0.38686(97) & 0.3572(13) & 0.3380(14) & 0.4655(13) & 0.4071(28) & 0.4365(18) \\\\', '    \\hline', '    Aaron     & 0.2969(18)   & 0.38497(69)    & 0.3560(12)      & 0.3360(11)     & 0.4582(17)   & 0.3999(38)   & 0.4289(27)         \\\\', '    \\hline', '    Daniel & 0.2986(19) & 0.38548(91) & 0.3562(14) & 0.3373(13) & 0.4608(15) & 0.4019(37) & 0.4316(22) \\\\ ', '    \\hline']
a06m220L
['\\begin{tabular}{|c|c|c|c|c|c|c|c|c|}', '    \\hline', '    a06m310L & $N_{E0}$ & $\\Xi_{EO}$ & $\\Sigma_{E0}$ & $\\Lambda_{E0}$ & $\\Xi^*_{E0}$ & $\\Delta_{E0}$ & $\\Sigma^*_{E0}$ \\\\', '    \\hline', '    Amy & {0.3265

In [12]:

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def plot_ensemble(ensemble_data, ensemble_name,pdf):
    fig, ax = plt.subplots(figsize=(12, 8))

    offset = 0.05

    for i, (person, results) in enumerate(ensemble_data.items()):
        if person == '\\hline':
            continue
        valid_results = {k: v for k, v in results.items() if len(v) > 0}
        x = np.arange(len(valid_results))
        y = [result[0] for result in valid_results.values()]
        yerr = [result[1] *0.0001 if len(result) > 1 else 0 for result in valid_results.values()]
        ax.errorbar(x + i * offset, y, yerr=yerr, fmt="o", capsize=5, label=person)

    ax.set_xticks(x + (len(ensemble_data) - 1) * offset / 2)
    ax.set_xticklabels(valid_results.keys())
    ax.set_xlim(-0.5, len(valid_results) - 0.5 + (len(ensemble_data) - 1) * offset)

    ax.legend()
    ax.set_xlabel("Baryon")
    ax.set_ylabel("Fit Result")
    ax.set_title(f"Fit Results for {ensemble_name}")

    pdf.savefig(fig)
    plt.close(fig)

with PdfPages("output_plots.pdf") as pdf:
    for ensemble_name, ensemble_data in data.items():
        plot_ensemble(ensemble_data, ensemble_name, pdf)


